In [49]:
# 数据透视表学习目标
# 1、什么是透视表
# 2、掌握Pandas透视表（pivot_table）使用方法

# 基本概念：数据透视表（Pivot Table）是一种交互式的表，可以进行某些计算，如求和与计数等。所进行的计算与数据跟数据透视表中的排列有关，之所以称为数据透视表，是因为可以动态地改变它们的版面布置，以便按照不同方式分析数据，也可以重新安排行号、列标和页字段。每一次改变版面布置时，数据透视表会立即按照新的布置重新计算数据。另外，如果原始数据发生更改，则可以更新数据透视表。

# 会销比 = 会员消费的金额 / 全部客户消费的金额 （会销比 = 会员消费的订单数 / 全部销售订单数）
#   -作用：统计的是会员消费占所有销售金额的比例，通过会销比可以衡量会员的整体质量

# 连带率 = 消费数量 / 订单数量
#   -作用: 通过连带率分析可以反映出人、货、场几个角度的业务问题 eg: 连带率直接影响到客单价;连带率反应运营质量

# 复购率 = 一段时间内消费次数大于1次的人数 / 总消费人数
#   -作用：指会员对该品牌产品或者服务的重复购买次数，重复购买率越多，则反应出会员对品牌的忠诚度就越高，反之则越低。通过复购率分析可以反映出运营状态


In [4]:
# 导入Pandas包
import pandas as pd

# 导入datetime包
from datetime import datetime

# 导入matplotlib包
import matplotlib.pyplot as plt

In [5]:
# 1、加载数据
customer_info = pd.read_excel('../data/会员信息查询.xlsx')
customer_info.info()

In [6]:
customer_info.info()

In [7]:
#会员信息查询
customer_info.head()

In [8]:
# 会员信息表 添加年月列
customer_info.loc[:, '注册年月'] = customer_info['注册时间'].apply(lambda x: x.strftime('%Y-%m'))
customer_info[['会员卡号', '会员等级', '会员来源', '注册时间', '注册年月']].head()

In [9]:
# 根据注册时间分组，获取每月新注册数量
month_count = customer_info.groupby('注册年月')[['会员卡号']].count()
month_count.columns = ['月增量']
month_count.head()

In [10]:
# 用数据透视表实现相同功能：dataframe.pivot_table()
# values: 要做聚合操作的列名
# index：行索引，传入原始数据的列名 原始数据的哪一个列作为新生成df中行索引
# columns：列索引，传入原始数据的列名 原始数据的哪一个列作为新生成df中列名
# aggfunc：聚合函数
customer_info.pivot_table(index='注册年月', values='会员卡号', aggfunc='count')

In [11]:
# 计算存量客户cumsum对某一列 做累积求和
month_count.loc[:, '存量'] = month_count['月增量'].cumsum()
month_count

In [12]:
# 设置正常显示汉字和负号
plt.rcParams['font.sans-serif'] = ['SimHei']  # 正常显示汉字
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号

In [13]:
# 可视化，需要去除第一个月数据，第一个月数据是之前所有会员数量的累积（数据质量问题）
month_count['月增量'].plot(figsize=(20, 8), color='yellow', secondary_y=True)
month_count['存量'].plot.bar(figsize=(20, 8), color='green', xlabel='年月', legend=True, ylabel='存量')
plt.title('会员存量增量分析', fontsize=20, fontweight='bold')

In [14]:
# 增量等级分布
columns_dict = ['注册年月', '会员等级']
month_degree_count = customer_info.groupby(columns_dict)[['会员卡号']].count()
month_degree_count

In [15]:
#分组之后得到的是multiIndex类型的索引，将multiIndex索引变成普通索引 使用reset_index()
month_degree_count.reset_index()

In [16]:
#使用unstack()
month_degree_count.unstack()


In [17]:
# 使用透视表实现数据汇总
member_rating = customer_info.pivot_table(index='注册年月', columns='会员等级', values='会员卡号', aggfunc='count')
member_rating

In [18]:
# 去掉第一个月的异常数据
member_rating = member_rating[1:]

# 1、创建一个画布（figsize：横纵坐标比；dpi：像素个数）
fig, ax1 = plt.subplots(figsize=(20, 8), dpi=100)

# 2、创建并返回一个共享x轴的子图
ax2 = ax1.twinx()

# 3、获取白银、黄金会员列数据，绘制一个分组条形图（ax:x轴；rot:旋转刻度标签（rotation缩写，取值0-360）；grid:显示刻度背景网格；xlabel:x轴标签；ylabel:y轴标签）
data_bar = member_rating[['白银会员', '黄金会员']]
data_bar.plot.bar(ax=ax1, rot=0, grid=True, xlabel='年月', ylabel='白银黄金')
# 在图上标明一个图例，用于说明每条曲线的文字显示（loc：位置参数）
ax1.legend(loc='upper left')

# 4、获取钻石、铂金会员列数据
data_polyline = member_rating[['钻石会员', '铂金会员']]
data_polyline.plot(ax=ax2, color=['red', 'gray'], ylabel='铂金钻石')
# 在图上标明一个图例，用于说明每条曲线的文字显示（loc：位置参数）
ax2.legend(loc='upper right')

# 5、绘图标题
plt.title('会员增量等级分布', fontsize=20)

In [19]:
# 1、加载各地区会销比数据

# 会员销售数据
customer_consume = pd.read_excel('../data/会员消费报表.xlsx')
print(customer_consume.head())

# 销售订单数量
sale_order_num = pd.read_excel('../data/全国销售订单数量表.xlsx')
print(sale_order_num.head())

# 门店信息表
store_info = pd.read_excel('../data/门店信息表.XLSX')
print(store_info.head())


In [20]:
# 会员消费报表添加年月列

# 1、获取订单日期
order_time = pd.to_datetime(customer_consume['订单日期'])

# 2、对日期进行格式化并转化为int类型
int_order_time = order_time.apply(lambda x: datetime.strftime(x, '%Y%m')).astype(int)

# 3、格式化后给数据添加新列
customer_consume.loc[:, '年月'] = int_order_time
customer_consume

In [21]:
# 为会员消费报表添加地区编码

# 1、获取店铺信息的关联字段
store_columns = store_info[['店铺代码', '地区编码']]

# 2、合并客户消费和店铺数据
merge_customer_consume_store = pd.merge(customer_consume, store_columns, on='店铺代码')
merge_customer_consume_store

In [22]:
# 剔除电商数据，统计会员购买订单数量

# 1、剔除类型为电销地区数据
eliminate_area_cc = merge_customer_consume_store[merge_customer_consume_store['地区编码'] != 'GBL6D01']

# 2、以地区编码为行索引, 年月为列索引，对消费数量进行求和聚合操作
member_orders = eliminate_area_cc.pivot_table(values='消费数量', index='地区编码', columns='年月', aggfunc='sum',
                                              margins=True)
member_orders

In [23]:
# 以地区代码为行索引，年月为列索引，对全部订单数进行求和的聚合操作
country_sales = sale_order_num.pivot_table(values = '全部订单数', index = '地区代码', columns = '年月', aggfunc = 'sum',margins = True)
country_sales

In [24]:
# 计算各地区会销比
result= member_orders / country_sales
result.map(lambda x: format(x,".2%"))

In [25]:
# 统计订单的数量：需要对"订单号"去重,并且只要"下单"的数据,"退单"的不要

# 1、根据筛选条件获取数据
order_data = merge_customer_consume_store.query(" 订单类型=='下单' & 地区编码!='GBL6D01'")

# 2、统计订单量需要去重  后面统计消费数量和消费金额不需要去重
order_count = order_data[['年月', '地区编码', '订单号']].drop_duplicates()

# 3、地区编码为行索引，年月为列索引，对订单号数量进行聚合求和
order_count = order_count.pivot_table(index = '地区编码', columns='年月', values='订单号', aggfunc='count')
order_count

In [26]:
# 统计消费商品数量
consume_count = order_data.pivot_table(values = '消费数量', index='地区编码', columns='年月', aggfunc='sum')
consume_count.head()


In [28]:
# 计算连带率
result = consume_count/order_count
#小数二位显示
result = result.map(lambda x:format(x,'.2f'))
result

In [44]:
# 会员复购率分析（由于一个会员同一天消费多次也算一次消费，所以会员消费次数按一天一次计算 因此需要对"会员卡号"和"时间"进行去重）

# 1、获取客户订单类型为下单，且下单时间在2018-01～2018-12
order_data = merge_customer_consume_store.query("订单类型 == '下单'")
order_data_condition = (order_data['年月'] <= 201812) & (order_data['年月'] >= 201801)
order_data = order_data[order_data_condition]

# 2、根据订单日期、卡号、年月、地区编码一起去重
order_data = order_data[['订单日期','卡号','年月','地区编码']].drop_duplicates()

# 3、行索引为地区编码和卡号，对订单日期进行求和聚合操作，形成透视图
consume_count = order_data.pivot_table(index = ['地区编码', '卡号'], values = '订单日期', aggfunc = 'count').reset_index()
# 字段名称进行替换
consume_count.rename(columns = {'订单日期':'消费次数'}, inplace = True)
consume_count

In [45]:
# 判断是否复购
consume_count['是否复购'] = consume_count['消费次数'] > 1
consume_count

In [46]:
# 计算复购率并定义函数（计算复购率并定义函数）
depart_data=consume_count.pivot_table(index = ['地区编码'], values=['消费次数','是否复购'], aggfunc={'消费次数':'count','是否复购':'sum'})
depart_data.columns=['复购人数','购买人数']
depart_data


In [47]:
depart_data.loc[:,'复购率']=depart_data['复购人数']/depart_data['购买人数']
depart_data


In [ ]:
# 小结
# 1、透视表是数据分析中经常使用的API，跟Excel中的数据透视表功能类似
# 2、Pandas的数据透视表，pivot_table，常用几个参数 index，values，columns，aggfuc，margin
# 3、Pandas的功能与groupby功能类似